In [55]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA 
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
import numpy as np
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics.pairwise import pairwise_distances_argmin

import warnings
warnings.filterwarnings("ignore")

In [56]:
df_=pd.read_csv(r'C:\Users\sna.tavakoli\Desktop\Sampling\Data_edited2.csv')

df=df_.drop(['id','year','message_code','service','atm_status',
       'atm_manufacturer','atm_location','atm_streetname',
       'atm_street_number','atm_zipcode','weather_lat',"atm_id",
       'weather_lon','weather_city_id','weather_city_name','weather_description'],axis=1)


df['rain_3h']=df['rain_3h'].fillna(0)
df.message_text=df.message_text.fillna("None")

df.message_text = df.message_text.replace(["Suspected malfunction, no cash dispensed","Suspected malfunction, card retained"],"Suspected malfunction")
df.message_text = df.message_text.replace(["Timed-out taking card, card retained and no cash dispensed","Timed-out taking money"],"Timed-out")

In [57]:
def faziday(x):
    a=0
    b=0
    c=0

    if x<4:
        a=1
    elif x<9:
        Y=x-4
        a = 1-Y*0.25
    elif x>8:
        a=0
        
    if x<4:
        b=0
    elif x<8:
        T = x-4
        b=T*0.25
    elif x<25:
        b=1
    elif x<29:
        T = x-25
        b=1-T*0.25
        T+=1
        
    if x<25:
        c=0
    elif x<29:
        L=x-25
        c=0.25*L
    elif x>28:
        c=1
    
    
    return round(a*10),round(b*10),round(c*10)

def fazihour(x):
    a=0
    b=0
    c=0

    if x<6:
        a=1
    elif x<11:
        Y=x-5
        a = 1-Y*0.2
    elif x>10:
        a=0
        
    if x<6:
        b=0
    elif x<11:
        T = x-5
        b=T*0.2
    elif x<15:
        b=1
    elif x<21:
        T = x-15
        b=1-T*0.2
        
        
    if x<15:
        c=0
    elif x<20:
        L=x-15
        c=0.20*L
    elif x>20:
        c=1
    
    
    return round(a*10),round(b*10),round(c*10)

In [58]:
df["1day"]=1
df["2day"]=2
df["3day"]=3
df["1day"],df["2day"],df["3day"] = zip(*df.day.apply(lambda x:faziday(x)))
df.drop("day",axis=1, inplace=True)

df["1hour"]=1
df["2hour"]=2
df["3hour"]=3
df["1hour"],df["2hour"],df["3hour"] = zip(*df.hour.apply(lambda x:fazihour(x)))
df.drop("hour",axis=1, inplace=True)


dff = pd.get_dummies(df[["month","weekday","currency","card_type",'weather_main',"message_text"]],drop_first=True)

df = df.drop(["month","weekday","currency","card_type",'weather_main',"message_text"],axis=1)

df = pd.merge(df,dff,left_index=True,right_index=True)

MMS = MinMaxScaler()
MMS.fit(df)
df_norm = MMS.transform(df)
df_norm = pd.DataFrame(df_norm, columns=df.columns)
# df_norm.head()

pca_model = PCA(n_components=0.98)
pca_model.fit(df_norm)
df_pca = pca_model.transform(df_norm)
df_pca = pd.DataFrame(df_pca)


pca_ratio=pca_model.explained_variance_ratio_
pca_ratio

array([0.09111237, 0.08091678, 0.06965594, 0.06097135, 0.05105725,
       0.0488295 , 0.04634851, 0.04567278, 0.04465698, 0.04452921,
       0.03984915, 0.03743889, 0.03676162, 0.03562685, 0.03452043,
       0.03234547, 0.02596047, 0.02200391, 0.02007065, 0.01652769,
       0.01496215, 0.01336286, 0.01223224, 0.00961662, 0.00820711,
       0.00806818, 0.00726507, 0.00681358, 0.00520524, 0.0048252 ,
       0.00394886, 0.00357514])

In [60]:
p=0
for j in df_pca.columns:
    df_pca[j] = df_pca[j]*pca_ratio[p]
    p+=1
    
df_pca = df_pca*10

In [61]:
distrToTal = []

for j in df_pca.columns:
    distr = []
    arange = []
    pas = 0
    sta = -0.95
    for i in range(2000): 
        lenn =len(df_pca[j][df_pca[j]<sta])
        
        distr.append((lenn / 1250000) - pas)
        pas = np.sum(distr)
        
        arange.append(round(sta,2))
        sta +=0.02
        
        if sta>1.5:
            break
            
    distrToTal.append(distr)
distrToTal

[[0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0002216,
  0.0002936,
  0.0012584,
  0.0021488,
  0.0029464,
  0.0059928,
  0.0071223999999999975,
  0.009534400000000002,
  0.0144064,
  0.019004800000000002,
  0.017206399999999997,
  0.0196504,
  0.02539440000000001,
  0.026361599999999985,
  0.024309600000000015,
  0.02221679999999998,
  0.020262400000000014,
  0.019407199999999986,
  0.017190400000000022,
  0.015111199999999991,
  0.012289600000000012,
  0.010629599999999961,
  0.010301600000000022,
  0.009730400000000028,
  0.010926400000000003,
  0.010967199999999955,
  0.011714400000000014,
  0.012526399999999993,
  0.013284799999999986,
  0.012768000000000002,
  0.010519200000000006,
  0.009223200000000042,
  0.008069599999999955,
  0.007333599999999996,
  0.007231200000000049,
  0.008851200000000004,
  0.010323199999999977,
  0.012585599999999975,
  0.012368000000000046,
  0.016913599999999973,
  0.01973839999999999,
  0.020088800000000018,
  0.01959920000000004,
  0.0176944,
  0.0155415

In [62]:
def fitness(df):
    summy = []
    for j in df.columns:
        j = int(j)
        sc=0
        pas=0
        count=0
        summ=[]
        for i in arange:
            lenn =df[df[j]<i].shape[0]
            pers = (lenn / df.shape[0]) - pas
            pas += pers
            summ.append(np.abs(pers - distrToTal[j][count]))
            count+=1
        summy.append(np.sum(summ) * pca_ratio[j])
        
    return np.sum(summy)

In [22]:
df_pca_mi = df_pca.copy()
mbkm =MiniBatchKMeans(n_clusters=1700,max_iter=800, batch_size=50000 , verbose=1)

mbkm.fit(df_pca_mi)
df_pca_mi["lable"] = mbkm.predict(df_pca_mi)



list0 = []
for i in range(1900):
    dff = df_pca_mi[df_pca_mi["lable"] == i]
    list1 = np.random.choice(dff.index ,size = int(np.round((len(dff)/10))),replace=False)
    list0.append(list1)
    
    
    
hhh = []
for i in list0:
    for j in i:
        hhh.append(j)
        
        
        
print(len(hhh))


df_sample = df_pca[df_pca.index.isin(hhh)]

fitness(df_sample)

Init 1/3 with method: k-means++
Inertia for init 1/3: 4888.982612
Init 2/3 with method: k-means++
Inertia for init 2/3: 4869.625900
Init 3/3 with method: k-means++
Inertia for init 3/3: 4891.193640
Minibatch iteration 1/20000: mean batch inertia: 0.033306, ewa inertia: 0.033306 
Minibatch iteration 2/20000: mean batch inertia: 0.033189, ewa inertia: 0.033297 
Minibatch iteration 3/20000: mean batch inertia: 0.033214, ewa inertia: 0.033290 
Minibatch iteration 4/20000: mean batch inertia: 0.033041, ewa inertia: 0.033270 
Minibatch iteration 5/20000: mean batch inertia: 0.032949, ewa inertia: 0.033245 
Minibatch iteration 6/20000: mean batch inertia: 0.033081, ewa inertia: 0.033231 
Minibatch iteration 7/20000: mean batch inertia: 0.033080, ewa inertia: 0.033219 
Minibatch iteration 8/20000: mean batch inertia: 0.033079, ewa inertia: 0.033208 
Minibatch iteration 9/20000: mean batch inertia: 0.032495, ewa inertia: 0.033151 
Minibatch iteration 10/20000: mean batch inertia: 0.033181, ewa 

0.009907310733614554

In [23]:
lili = []
for i in range(len(arange)-1):
    pp = len(df_pca[df_pca[0]<arange[i+1]]) - len(df_pca[df_pca[0]<arange[i]])
    lili.append(round(pp/10))
    print(round(pp/10))

0
0
0
0
28
37
157
269
368
749
890
1192
1801
2376
2151
2456
3174
3295
3039
2777
2533
2426
2149
1889
1536
1329
1288
1216
1366
1371
1464
1566
1661
1596
1315
1153
1009
917
904
1106
1290
1573
1546
2114
2467
2511
2450
2212
1943
1599
1273
1173
1038
928
737
1063
1067
1012
1200
1537
1513
1464
1717
1521
1228
1020
832
694
502
386
326
284
314
338
310
652
847
1024
1182
1492
1629
1822
1711
1620
1498
1313
1066
737
697
669
590
426
405
606
611
531
559
708
645
576
653
626
494
310
381
410
302
201
124
84
44
17
5
2
0
0
0
0
0
0
0
0


In [25]:
from tqdm import tqdm
best=10
for i in tqdm(range(100)):
    ind = []

    for i in range(len(arange)-1):
        data = df_pca[(df_pca[0]<arange[i+1]) & (df_pca[0]>arange[i])]
        data = data.iloc[:,1:]
        le = round(len(data)/10)

        if le<500:
            tt = np.random.choice(data.index,le,replace=False)
            ind.append(tt)
        else:
            n_c = round(le/100)
            mbkm =MiniBatchKMeans(n_clusters=n_c,max_iter=800, batch_size=round(le/4) , verbose=0)

            mbkm.fit(data)
            data["lable"] = mbkm.predict(data)



            list3 = []
            for i in range(n_c):
                dff = data[data["lable"] == i]
                list1 = np.random.choice(dff.index ,size = int(np.round((len(dff)/10))),replace=False)
                list3.append(list1)

            hhh = []
            for i in list3:
                for j in i:
                    hhh.append(j)

            ind.append(hhh)
    fina = []
    for i in ind:
        for j in i:
            fina.append(j)
            
    df_sample = df_pca[df_pca.index.isin(fina)] 
    fit = fitness(df_sample)

    if fit<best:
        best = fit
        best_sample=df_sample
        print(1-best)

  1%|▊                                                                               | 1/100 [01:05<1:48:23, 65.70s/it]

0.009466405620333367


  3%|██▍                                                                             | 3/100 [03:15<1:45:50, 65.47s/it]

0.00943097750856115


  6%|████▊                                                                           | 6/100 [06:30<1:41:54, 65.05s/it]

0.009177036434876319


 51%|█████████████████████████████████████████▎                                       | 51/100 [55:24<53:13, 65.16s/it]

0.008993309732529924


100%|██████████████████████████████████████████████████████████████████████████████| 100/100 [1:48:28<00:00, 65.08s/it]


In [28]:
naser = 201
best=0.05
for i in tqdm(range(100)):
    ind = []

    for i in range(len(arange)-1):
        data = df_pca[(df_pca[0]<arange[i+1]) & (df_pca[0]>arange[i])]
        data = data.iloc[:,1:]
        le = round(len(data)/naser)
#         print(le)
        if le<100:
            tt = np.random.choice(data.index,le,replace=False)
            ind.append(tt)
        else:
            n_c = round(le/naser)
            if n_c<1:
                n_c=1
            mbkm =MiniBatchKMeans(n_clusters=n_c,max_iter=800, batch_size= n_c, verbose=0)

            mbkm.fit(data)
            data["lable"] = mbkm.predict(data)



            liste = []
            for i in range(n_c):
                dff = data[data["lable"] == i]
                list1 = np.random.choice(dff.index ,size = int(np.round((len(dff)/naser))),replace=False)
                liste.append(list1)

            hhh = []
            for i in liste:
                for j in i:
                    hhh.append(j)

            ind.append(hhh)
    fina = []
    for i in ind:
        for j in i:
            fina.append(j)
            
    df_sample = df_pca[df_pca.index.isin(fina)] 
    fit = fitness(df_sample)
#     print(fit)
    if fit>best:
        print(1250000/naser)
        break
    naser +=1
    print(naser-1)
    print(1 - fit)
    print(len(fina))

  1%|▊                                                                               | 1/100 [01:32<2:32:55, 92.69s/it]

201
0.9509892512491512
6214


  2%|█▌                                                                              | 2/100 [03:09<2:33:23, 93.91s/it]

202
0.9505885396802707
6181


  3%|██▍                                                                             | 3/100 [04:40<2:30:29, 93.09s/it]

...

204
0.9506226384693253
6123


  5%|████                                                                            | 5/100 [07:48<2:28:16, 93.65s/it]

205
0.9521001862482276
6099


  5%|███▉                                                                           | 5/100 [09:26<2:59:26, 113.33s/it]

6067.961165048544


In [40]:
hihi = pd.read_csv(r"C:\Users\sna.tavakoli\Desktop\Sampling\mojahedi\genetics_sample.csv")
rtrt = list(hihi["original_index"])
df_samplen = pd.DataFrame()
df_samplen = df_pca[df_pca.index.isin(rtrt)]

error2 = fitness(df_samplen)
1 - error2

0.9458889517290141

In [39]:
lo = 10
for i in range(100):
    lo +=lo

In [74]:
best_sample.to_csv(r"C:\Users\sna.tavakoli\Desktop\Sampling\final\final125000.csv")

df_pca[df_pca.index.isin(fina)].to_csv(r"C:\Users\sna.tavakoli\Desktop\Sampling\final\final6069.csv")